In [6]:
from model import *
from data import *
from metrics import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 
import segmentation_models as sm

%load_ext autoreload
%autoreload 2

Segmentation Models: using `tf.keras` framework.


In [7]:
BACKBONE = 'resnext50' #Pretrained backbone

In [8]:
data_gen_args = dict(rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=[0.8,1],
                    brightness_range=[0.9,1.1],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')

In [9]:
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(16,'dataset','images_satellite','labels_satellite',data_gen_args)
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [8]:
model = sm.Unet(BACKBONE,encoder_weights='imagenet',input_shape=(256,256,3),encoder_freeze=True)
model.compile(optimizer = Adam(lr = 5e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

In [9]:
model_checkpoint = ModelCheckpoint('pretrained_unet_final.hdf5', monitor='val_mean_iou_weighted',verbose=1, save_best_only=True, mode = 'max')
history1=model.fit_generator(train_generator,steps_per_epoch=400,epochs=15,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
400/400 [==============================] - ETA: 0s - loss: 0.0102 - accuracy: 0.7089 - mean_iou: 0.6000 - mean_iou_neg: 0.6996 - mean_iou_weighted: 0.6797 - dice: 0.7255 - dice_neg: 0.8025 - dice_weighted: 0.7871
Epoch 00001: val_mean_iou_weighted improved from -inf to 0.27759, saving model to pretrained_unet_final.hdf5
400/400 [==============================] - 239s 598ms/step - loss: 0.0102 - accuracy: 0.7089 - mean_iou: 0.6000 - mean_iou_neg: 0.6996 - mean_iou_weighted: 0.6797 - dice: 0.7255 - dice_neg: 0.8025 - dice_weighted: 0.7871 - val_loss: 0.0572 - val_accuracy: 0.6264 - val_mean_iou: 5.1571e-11 - val_mean_iou_neg: 0.3470 - val_mean_iou_weighted: 0.2776 - val_dice: 5.1571e-11 - val_dice_neg: 0.4005 - val_dice_weighted: 0.3204
Epoch 2/15
400/400 [==============================] - ETA: 0s - loss: 0.0064 - accuracy: 0.7374 - mean_iou: 0.7005 - mean_iou_neg: 0.7712 - mean_iou_weighted: 0.7571 - 

Epoch 12/15
400/400 [==============================] - ETA: 0s - loss: 0.0030 - accuracy: 0.7653 - mean_iou: 0.8058 - mean_iou_neg: 0.8528 - mean_iou_weighted: 0.8434 - dice: 0.8857 - dice_neg: 0.9152 - dice_weighted: 0.9093
Epoch 00012: val_mean_iou_weighted did not improve from 0.64375
400/400 [==============================] - 230s 575ms/step - loss: 0.0030 - accuracy: 0.7653 - mean_iou: 0.8058 - mean_iou_neg: 0.8528 - mean_iou_weighted: 0.8434 - dice: 0.8857 - dice_neg: 0.9152 - dice_weighted: 0.9093 - val_loss: 0.0129 - val_accuracy: 0.7041 - val_mean_iou: 0.5268 - val_mean_iou_neg: 0.6694 - val_mean_iou_weighted: 0.6409 - val_dice: 0.6504 - val_dice_neg: 0.7704 - val_dice_weighted: 0.7464
Epoch 13/15
400/400 [==============================] - ETA: 0s - loss: 0.0027 - accuracy: 0.7641 - mean_iou: 0.8089 - mean_iou_neg: 0.8550 - mean_iou_weighted: 0.8458 - dice: 0.8886 - dice_neg: 0.9171 - dice_weighted: 0.9114
Epoch 00013: val_mean_iou_weighted improved from 0.64375 to 0.64716, sa

In [10]:
save_obj(history1.history,'pretrained_history')

In [13]:
model_checkpoint = ModelCheckpoint('pretrained_unet_final_resume.hdf5', monitor='val_mean_iou_weighted',verbose=1, save_best_only=True, mode = 'max')

for layer in model.layers:
    if not isinstance(layer,tf.keras.layers.BatchNormalization):
        layer.trainable = True
model.compile(optimizer = Adam(lr = 1e-6), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])
history2=model.fit_generator(train_generator,steps_per_epoch=400,epochs=5,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

Epoch 1/5
400/400 [==============================] - ETA: 0s - loss: 0.0020 - accuracy: 0.7681 - mean_iou: 0.8236 - mean_iou_neg: 0.8657 - mean_iou_weighted: 0.8573 - dice: 0.8996 - dice_neg: 0.9247 - dice_weighted: 0.9196
Epoch 00001: val_mean_iou_weighted improved from -inf to 0.65087, saving model to pretrained_unet_final_resume.hdf5
400/400 [==============================] - 256s 641ms/step - loss: 0.0020 - accuracy: 0.7681 - mean_iou: 0.8236 - mean_iou_neg: 0.8657 - mean_iou_weighted: 0.8573 - dice: 0.8996 - dice_neg: 0.9247 - dice_weighted: 0.9196 - val_loss: 0.0122 - val_accuracy: 0.7135 - val_mean_iou: 0.5374 - val_mean_iou_neg: 0.6792 - val_mean_iou_weighted: 0.6509 - val_dice: 0.6625 - val_dice_neg: 0.7798 - val_dice_weighted: 0.7564
Epoch 2/5
400/400 [==============================] - ETA: 0s - loss: 0.0019 - accuracy: 0.7667 - mean_iou: 0.8249 - mean_iou_neg: 0.8670 - mean_iou_weighted: 0.8585 - dice: 0.9000 - dice_neg: 0.9251 - dice_weighted: 0.9200
Epoch 00002: val_mean_i

In [14]:
save_obj(history2.history,'pretrained_resume_history')

In [5]:
test_img, names = test('dataset/test_images')

folder = './dataset/test_labels'
num_img, size = test_img.shape[0], test_img.shape[1]            

test_label = generate_labels(folder, names, num_img, size)

In [16]:
model = load_model('pretrained_unet_final_resume.hdf5', compile=False)
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy, mean_iou_, mean_iou_neg_, mean_iou_weighted_, dice_, dice_neg_, dice_weighted_ = model.evaluate(test_img, test_label)
print('Test score:', score)

1/1 [==============================] - 0s 1ms/step - loss: 0.0717 - accuracy: 0.8370 - mean_iou: 0.1370 - mean_iou_neg: 0.4824 - mean_iou_weighted: 0.4133 - dice: 0.2291 - dice_neg: 0.5664 - dice_weighted: 0.4990
Test score: 0.07167227566242218


In [4]:
data_gen_args = dict(rotation_range=30,
                    horizontal_flip=True,
                    vertical_flip=True)


test_gen = trainGenerator_simple(4,'dataset','test_images','test_labels', aug_dict = data_gen_args, image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 1,save_to_dir = None,
                   target_size = (256,256),seed = 1)

model = load_model('pretrained_unet_final.hdf5',  compile = False, custom_objects={'binary_focal_loss_fixed':binary_focal_loss()})
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy,  iou, iou_neg, iou_weighted, dice_coeff, dice_coeff_neg, dice_coeff_weighted= model.evaluate(test_gen,steps=20)

Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
20/20 [==============================] - 2s 96ms/step - loss: 0.0684 - accuracy: 0.8460 - mean_iou: 0.1347 - mean_iou_neg: 0.4861 - mean_iou_weighted: 0.4158 - dice: 0.2178 - dice_neg: 0.5636 - dice_weighted: 0.4945
